In [1]:
#Install the nessasary packages from pip
!pip install mediapipe opencv-python pandas scikit-learn --user

In [1]:
#Import all the nessacary libraries from the packages

#Import mediapipe to be use as the model
import mediapipe as mp
#Import opencv for rendaring and drawing capabilities
import cv2

import numpy as np #Handle numpy arrays
import pandas as pd #Handle tabular data
import csv #Handle csv files
import os #Handle folder structure
import glob
import pickle #Save and oad ML model

from sklearn.model_selection import train_test_split #Partition the data into training and testing

from sklearn.pipeline import make_pipeline #Creates a pipeline
from sklearn.preprocessing import StandardScaler #Standadize data 

#Classification algorithms
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score #Evaluate model through accuracy

In [2]:
#Helper to draw the landmarks and provide the landmark detection models
draw_helpers = mp.solutions.drawing_utils 
holistic_model = mp.solutions.holistic

In [3]:
#Number of landmarks considered.
num_landmarks = 33

In [4]:
#Save the landmarks to a table to be exported as a csv file

#0th column of the table
table_columns = ['class']
#Add columns to the table according to the no.of landmarks
for num in range(1, num_landmarks + 1):
    table_columns += ['x{}'.format(num), 'y{}'.format(num), 'z{}'.format(num), 'v{}'.format(num)]
    
#Display columns of the table
table_columns

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

# Creating the csv file for store extracted data

In [5]:
csv_file_pth = 'data.csv'

#Write to the csv file
with open(csv_file_pth, mode='w', newline='') as f:
    #Define the csv writer
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(table_columns)
    print("File Created")

File Created


# Function to extract data from video and store inside the CSV

In [15]:

def video_processor(class_name, video_pth):
    #Connect the sample video from the device
    sample_video = cv2.VideoCapture(video_pth)
    processed = False


    #Load the holistic model
    with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        #Loop through each frame of the video 
        while sample_video.isOpened():
            #Returns the status of the read and the frame as an image
            status, frame = sample_video.read()

            #If frame is read correctly, status is true
            if status == False:
                break

            #Recolor the captured frame from BGR to RGB (Medipipe requies frames to be in RGB format)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            #Prevent writing and copying frame data to improve performance while making the detection
            rgb_frame.flags.writeable = False        

            #Use holistic model to make detections
            result_frame = holistic.process(rgb_frame)

            #Set frame back to writable format after detection
            rgb_frame.flags.writeable = True   

            #Recolor the captured frame from BGR for rendering with opencv
            bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

            #Use pose model to detect only the landmarks of the the body and not the landmarks of the face and hand
            draw_helpers.draw_landmarks(bgr_frame, result_frame.pose_landmarks, holistic_model.POSE_CONNECTIONS, 
                                 draw_helpers.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 draw_helpers.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
         

            #Export the coordinates of the landmarks to the csv file
            try:
                #Extracting all the landmarks of the pose as an array
                pose_landmarks_array = result_frame.pose_landmarks.landmark
                #Format landmarks in to a numpy array for better structuring(removing keys) and collapse array to 1 dimesnsion
                pose_landmarks_nparray = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_landmarks_array]).flatten() 
                                              if result_frame.pose_landmarks else np.zeros(33*4))

                #Append class name as the Oth element
                pose_landmarks_nparray.insert(0, class_name)

                #Append the data to table in the csv file
                with open(csv_file_pth, mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(pose_landmarks_nparray) 
                
                processed = True
            except:
                pass
            
            #Display the frames    
            cv2.imshow('Results Feed', bgr_frame)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    
    
    sample_video.release()
    cv2.destroyAllWindows()
    
    #If the try block is processed without any error
    if processed:
        print("Processed")

In [16]:
#Specify the location to the datasets
valid_path = "./datasets/valid/*.mp4"
invalid_path = "./datasets/invalid/*.mp4"

#Displat the no of files in the datasets
print("Valid Video Count: ", len(glob.glob(valid_path)))
print("Invalid Video Count: ", len(glob.glob(invalid_path)))

Valid Video Count:  1
Invalid Video Count:  6


In [17]:
#Adding landmarks of the invalid dataset to the csv
class_name = "correct"
dir_size = len(glob.glob(valid_path))
for i in range (1, dir_size + 1):
    video_pth = "./datasets/valid/" + str(i) + ".mp4"
    print("Video: ", str(i), "/", str(dir_size))
    video_processor(class_name, video_pth)

Video:  1 / 1
Processed


In [19]:
#Adding landmarks of the invalid dataset to the csv
class_name = "Incorrect"
dir_size = len(glob.glob(invalid_path))
for i in range (1, dir_size + 1):
    video_pth = "./datasets/invalid/" + str(i) + ".mp4"
    print("Video: ", str(i), "/", str(dir_size))
    video_processor(class_name, video_pth)

Video:  1 / 6
Processed
Video:  2 / 6
Processed
Video:  3 / 6
Processed
Video:  4 / 6
Processed
Video:  5 / 6
Processed
Video:  6 / 6
Processed


# Customize data read from the csv file

In [20]:
#Import dataframe
df = pd.read_csv(csv_file_pth)

In [21]:
#Display the first 5 rows in the dataframe
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,correct,0.514309,0.324966,-0.617117,0.999996,0.526141,0.313048,-0.572653,0.999993,0.534911,...,-0.178787,0.936461,0.618069,0.768465,-0.311516,0.967621,0.437434,0.761638,-0.402793,0.980149
1,correct,0.515676,0.325378,-0.654440,0.999996,0.526780,0.313310,-0.608953,0.999993,0.534889,...,-0.192884,0.932096,0.617116,0.768408,-0.426562,0.967040,0.446999,0.759817,-0.416264,0.978834
2,correct,0.516491,0.325497,-0.660133,0.999996,0.527250,0.313349,-0.614741,0.999993,0.534882,...,-0.185767,0.930318,0.616888,0.768206,-0.383256,0.966650,0.446460,0.759000,-0.402185,0.978387
3,correct,0.516632,0.325768,-0.667201,0.999996,0.527409,0.313483,-0.621792,0.999994,0.534833,...,-0.171473,0.928383,0.616794,0.768200,-0.368450,0.966315,0.446914,0.758990,-0.385919,0.977948
4,correct,0.516622,0.326694,-0.667572,0.999996,0.527422,0.314225,-0.623007,0.999994,0.534748,...,-0.170069,0.927042,0.616544,0.768161,-0.366961,0.965806,0.447594,0.758751,-0.382823,0.977470


In [22]:
#Display the last 5 rows in the dataframe
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
17472,Incorrect,0.522268,0.324088,-0.554544,0.999999,0.531129,0.312152,-0.501417,0.999999,0.537841,...,-0.692036,0.927402,0.705624,0.761274,-0.894458,0.971847,0.455574,0.762870,-0.995704,0.973596
17473,Incorrect,0.522241,0.324008,-0.553307,0.999999,0.531124,0.312075,-0.500169,0.999999,0.537840,...,-0.694852,0.927831,0.705841,0.761208,-0.895237,0.972113,0.455630,0.762888,-0.998968,0.973961
17474,Incorrect,0.522241,0.323894,-0.552821,0.999999,0.531125,0.311965,-0.499861,0.999999,0.537846,...,-0.694272,0.928380,0.706021,0.761151,-0.895362,0.972442,0.455632,0.762903,-0.997610,0.974379
17475,Incorrect,0.522236,0.323809,-0.547936,0.999999,0.531127,0.311881,-0.495483,0.999999,0.537851,...,-0.693655,0.928732,0.706137,0.761120,-0.895389,0.972685,0.455632,0.762906,-0.995620,0.974648
17476,Incorrect,0.522217,0.323700,-0.542424,0.999999,0.531127,0.311770,-0.490039,0.999999,0.537857,...,-0.693568,0.929211,0.706248,0.761071,-0.895468,0.972919,0.455579,0.762970,-0.995406,0.974933


In [23]:
#Remove the class column so the dataframe only contains features
X = df.drop('class', axis=1)
#Use the class as the target value
Y = df['class'] 

In [24]:
#Split the data with 30% for testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1234)

# Make predictions and select the best classifier

In [25]:
#Setup the machine learning model pipelines
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(max_iter=20000)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [26]:
#Dictionary to store the label of the model and model after training
train_models = {}
for label, pipeline in pipelines.items():
    model = pipeline.fit(X_train.values, Y_train.values)
    train_models[label] = model

In [27]:
train_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(max_iter=20000))]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [28]:
#Test the accuracies of the model to choose the best classifier
for label, model in train_models.items():
    output_class = model.predict(X_test.values)
    print(label, accuracy_score(Y_test.values, output_class))

lr 0.9952326468344775
rc 0.9794050343249427
rf 1.0
gb 0.9992372234935164


In [29]:
#Select and dump the classifier into a pickle file
model = train_models['rf']
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [30]:
#Save the model as a binary file
with open('sholder_press.pkl', 'wb') as f:
    pickle.dump(model, f)

In [31]:
#Import the model from the binary file
with open('sholder_press.pkl', 'rb') as f:
    model = pickle.load(f)

In [32]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [33]:
#PREDICT AND DISPLAY THE RESULTS OF THE MODEL BY PASSING THE TEST VIDEO

#Connect the test video from the device
sample_video = cv2.VideoCapture('datasets/IMG_0126.MOV')

#Load the holistic model
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through each frame of the video 
    while sample_video.isOpened():
        #Returns the status of the read and the frame as an image
        status, frame = sample_video.read()
        
        #If frame is read correctly, status is true
        if status == False:
            print("Done")
            break
          
        #Recolor the captured frame from BGR to RGB (Medipipe requies frames to be in RGB format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Prevent writing and copying frame data to improve performance while making the detection
        rgb_frame.flags.writeable = False        
        
        #Use holistic model to make detections
        result_frame = holistic.process(rgb_frame)
        
        #Set frame back to writable format after detection
        rgb_frame.flags.writeable = True   
        
        #Recolor the captured frame from BGR for rendering with opencv
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        #Use pose model to detect only the landmarks of the the body and not the landmarks of the face and hand
        draw_helpers.draw_landmarks(bgr_frame, result_frame.pose_landmarks, holistic_model.POSE_CONNECTIONS, 
                                 draw_helpers.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 draw_helpers.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
         #Predict the coordinates of the landmarks (resulrs screen)
        try:
            #Extracting all the landmarks of the pose as an array
            pose_landmarks_array = result_frame.pose_landmarks.landmark
            #Format landmarks in to a numpy array for better structuring(removing keys) and collapse array to 1 dimesnsion
            pose_landmarks_nparray = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_landmarks_array]).flatten())

            #Pass the numpy array into a data frame
            features = pd.DataFrame([pose_landmarks_nparray])
            
            #Store the top class of the prediction
            pose_class_status = model.predict(features.values)[0]
            #Store the probability of the prediction
            pose_class_status_prob = model.predict_proba(features.values)[0]
            
            print("Class:", pose_class_status)
            print(pose_class_status_prob)
            
            #Set a rectangle box to display the results of the prediction in the video frame
            #rectangle(container, top_coord, bottom_coord, color, line_thickness)
            cv2.rectangle(bgr_frame, (0,0), (250, 60), (245, 117, 16), -1)
            
            #Display the class label inside the rectangle box
            cv2.putText(bgr_frame, 'Class'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract =and display the top class of the prediction
            cv2.putText(bgr_frame, pose_class_status.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Display the class probability inside the rectangle box
            cv2.putText(bgr_frame, 'Probability'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract and dispthe maximum probability
            cv2.putText(bgr_frame, str(round(pose_class_status_prob[np.argmax(pose_class_status_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
 
        except:
            pass
                        
        #Display the frames    
        cv2.imshow('Results Feed', bgr_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

sample_video.release()
cv2.destroyAllWindows()

Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.91 0.09]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.91 0.09]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.88 0.12]
Class: Incorrect
[0.88 0.12]
Class: Incorrect
[0.88 0.12]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.88 0.12]
Class: Incorrect
[0.87 0.13]
Class: Incorrect
[0.87 0.13]
Class: Incorrect
[0.87 0.13]
Class: Incorrect
[0.87 0.13]
Class: Incorrect
[0.87 0.13]
Class: Incorrect
[0.87 0.13]
Class: Incorrect
[0.87 0.13]
Class: Incorrect
[0.87 0.13]
Class: Incorrect
[0.87 0.13]
Class: Incorrect
[0.88 0.12]
Cla

Class: correct
[0.44 0.56]
Class: correct
[0.44 0.56]
Class: correct
[0.44 0.56]
Class: correct
[0.43 0.57]
Class: correct
[0.43 0.57]
Class: correct
[0.43 0.57]
Class: correct
[0.43 0.57]
Class: correct
[0.43 0.57]
Class: correct
[0.43 0.57]
Class: correct
[0.43 0.57]
Class: correct
[0.43 0.57]
Class: correct
[0.42 0.58]
Class: correct
[0.42 0.58]
Class: correct
[0.41 0.59]
Class: correct
[0.41 0.59]
Class: correct
[0.41 0.59]
Class: correct
[0.43 0.57]
Class: correct
[0.43 0.57]
Class: correct
[0.43 0.57]
Class: correct
[0.42 0.58]
Class: correct
[0.42 0.58]
Class: correct
[0.42 0.58]
Class: correct
[0.43 0.57]
Class: correct
[0.44 0.56]
Class: correct
[0.44 0.56]
Class: correct
[0.45 0.55]
Class: correct
[0.45 0.55]
Class: correct
[0.44 0.56]
Class: correct
[0.43 0.57]
Class: correct
[0.42 0.58]
Class: correct
[0.42 0.58]
Class: correct
[0.42 0.58]
Class: correct
[0.42 0.58]
Class: correct
[0.38 0.62]
Class: correct
[0.37 0.63]
Class: correct
[0.35 0.65]
Class: correct
[0.29 0.71]
C

Class: Incorrect
[0.61 0.39]
Class: Incorrect
[0.61 0.39]
Class: Incorrect
[0.63 0.37]
Class: Incorrect
[0.65 0.35]
Class: Incorrect
[0.65 0.35]
Class: Incorrect
[0.66 0.34]
Class: Incorrect
[0.67 0.33]
Class: Incorrect
[0.68 0.32]
Class: correct
[0.34 0.66]
Class: correct
[0.26 0.74]
Class: correct
[0.27 0.73]
Class: correct
[0.31 0.69]
Class: Incorrect
[0.6 0.4]
Class: Incorrect
[0.6 0.4]
Class: correct
[0.45 0.55]
Class: correct
[0.45 0.55]
Class: correct
[0.46 0.54]
Class: correct
[0.33 0.67]
Class: correct
[0.16 0.84]
Class: correct
[0.16 0.84]
Class: correct
[0.16 0.84]
Class: correct
[0. 1.]
Class: correct
[0. 1.]
Class: correct
[0. 1.]
Class: correct
[0. 1.]
Class: correct
[0. 1.]
Class: correct
[0. 1.]
Class: correct
[0.17 0.83]
Class: Incorrect
[0.52 0.48]
Class: Incorrect
[0.72 0.28]
Class: Incorrect
[0.75 0.25]
Class: Incorrect
[0.75 0.25]
Class: Incorrect
[0.76 0.24]
Class: Incorrect
[0.78 0.22]
Class: Incorrect
[0.78 0.22]
Class: Incorrect
[0.78 0.22]
Class: Incorrect
[0.

Class: Incorrect
[0.76 0.24]
Class: Incorrect
[0.8 0.2]
Class: Incorrect
[0.82 0.18]
Class: Incorrect
[0.85 0.15]
Class: Incorrect
[0.85 0.15]
Class: Incorrect
[0.84 0.16]
Class: Incorrect
[0.84 0.16]
Class: Incorrect
[0.87 0.13]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.9 0.1]
Class: Incorrect
[0.89 0.11]
Class: Incorrect
[0.87 0.13]
Class: Incorrect
[0.86 0.14]
Class: Incorrect
[0.77 0.23]
Class: Incorrect
[0.55 0.45]
Class: correct
[0.49 0.51]
Class: correct
[0.45 0.55]
Class: Incorrect
[0.53 0.47]
Class: correct
[0.42 0.58]
Class: correct
[0.4 0.6]
Class: correct
[0.41 0.59]
Class: correct
[0.4 0.6]
Class: correct
[0.38 0.62]
Class: correct
[0.36 0.64]
Class: Incorrect
[0.6 0.4]
Class: Incorrect
[0.58 0.42]
Class: Incorrect
[0.61 0.39]
Class: Incorrect
[0.6 0.4]
Class: Incorrect
[0.59 0.41]
Class: Incorrect
[0.58 0.42]
Class: correct
[0.35 0.65]
Class: correct
[0.4 0.6]
Class: correct
[0.41 0.

KeyboardInterrupt: 